In [1]:
import re
import csv
from collections import Counter, defaultdict

In [65]:
logfile = 'sample.log'
csv_output_file = 'log_analysis_results.csv'
failed_login_thresold = 10

In [66]:
ip_reg = r'(\d+\.\d+\.\d+\.\d+)'
endpoint_reg = r'"(?:GET|POST|PUT|DELETE|PATCH) (\/\S+)'
status_code= r'HTTP\/\d\.\d" (\d+)'
failed_login = '401'

In [67]:
ip_request_count = Counter()
endpoint_access_count = Counter()
failed_logins = defaultdict(int)

In [68]:
with open(logfile, 'r') as file:
    for line in file:
        ip_match = re.search(ip_reg, line)
        if ip_match:
            ip = ip_match.group(1)
            ip_request_count[ip] += 1

        endpoint_match = re.search(endpoint_reg, line)
        if endpoint_match:
            endpoint = endpoint_match.group(1)
            endpoint_access_count[endpoint] += 1

        status_code_match = re.search(status_code, line)
        if status_code_match and status_code_match.group(1) == failed_login:
            if ip_match:
                failed_logins[ip] += 1
sorted_ip_requests = ip_request_count.most_common()
most_accessed_endpoint = max(endpoint_access_count.items(), key=lambda x: x[1])
suspicious_ips = [(ip, count) for ip, count in failed_logins.items() if count > failed_login_thresold]
print("IP Address          Request Count")
for ip, count in sorted_ip_requests:
    print(f"{ip:<20}{count}")

print("\nMost Frequently Accessed Endpoint:")
print(f"{most_accessed_endpoint[0]} (Accessed {most_accessed_endpoint[1]} times)")
if suspicious_ips:
    print("\nSuspicious Activity Detected:")
    print("IP Address          Failed Login Attempts")
    for ip, count in suspicious_ips:
        print(f"{ip:<20}{count}")
else:
    print("\nNo suspicious activity detected.")


IP Address          Request Count
203.0.113.5         8
198.51.100.23       8
192.168.1.1         7
10.0.0.2            6
192.168.1.100       5

Most Frequently Accessed Endpoint:
/login (Accessed 13 times)

No suspicious activity detected.


In [69]:
with open(csv_output_file, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["Requests per IP"])
    writer.writerow(["IP Address", "Request Count"])
    writer.writerows(sorted_ip_requests)
    writer.writerow([])
    writer.writerow(["Most Accessed Endpoint"])
    writer.writerow(["Endpoint", "Access Count"])
    writer.writerow([most_accessed_endpoint[0], most_accessed_endpoint[1]])
    writer.writerow([])
    writer.writerow(["Suspicious Activity"])
    writer.writerow(["IP Address", "Failed Login Count"])
    writer.writerows(suspicious_ips)

print(f"\nResults saved to {csv_output_file }.")


Results saved to log_analysis_results.csv.
